In [33]:
import urllib.request, urllib.parse, urllib.error #Used to access , request and parse urls
import json # Used to parse the json file format found in the url 
import psycopg2 # Postgresql database adapter used in storing data into a postgresql database 

In [37]:
# Creating database and tables within the database 

conn = psycopg2.connect(
    host="localhost",
    database="f1_9",
    user="postgres",
    password="xxxxxx",
    port = 5433)

cur = conn.cursor()
cur.execute('''
Create Table If NOT exists Driver (
    id SERIAL PRIMARY KEY,
    driver_name TEXT Unique
 );

create Table if Not exists races (
    id SERIAL PRIMARY KEY,
    driver_id INTEGER,
    track VARCHAR,
    points INTEGER,
    position INTEGER,
    race_times VARCHAR,
    constructor VARCHAR
);
''')
conn.commit()


In [38]:
race_standings = range(1,11) # This is a range of the 10 positions interested in 
tracks = range(0,22) # This is the number of tracks that were in the season 

In [39]:
# API URL 
service_urls = list()
for i in race_standings:
    service_url = f"http://ergast.com/api/f1/2022/results/{i}.json"
    service_urls.append(service_url)

In [40]:
# Inserting specific values from the url to the database 

for i in service_urls:
    uh = urllib.request.urlopen(i)
    data = uh.read().decode()
    datas = json.loads(data)
    
    for r in tracks:
        driver_name = datas['MRData']['RaceTable']['Races'][r]['Results'][0]['Driver']['driverId']
        driver_id = datas['MRData']['RaceTable']['Races'][r]['Results'][0]['Driver']['permanentNumber']
        constructor_name = datas['MRData']['RaceTable']['Races'][r]['Results'][0]['Constructor']['name']
        position = datas['MRData']['RaceTable']['Races'][r]['Results'][0]['position']
        track =  datas['MRData']['RaceTable']['Races'][r]['Circuit']['circuitId']
        points =  datas['MRData']['RaceTable']['Races'][r]['Results'][0]['points']
        cur.execute('INSERT INTO Driver (id,driver_name) VALUES (%s,%s) ON CONFLICT DO NOTHING',(driver_id,driver_name))
        cur.execute('INSERT INTO races (driver_id, track, points,position, race_times, constructor) VALUES (%s,%s,%s,%s,%s,%s)',(driver_id,track,points, position, race_time,constructor_name))
        conn.commit()
